In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
from dataclasses import dataclass
from pprint import pprint
from prettytable import PrettyTable

2022-12-08 19:18:27.856380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 19:18:28.056382: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-08 19:18:28.800274: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/lib64:/usr/local/cuda-11/lib64:
2022-12-08 19:18:28.800539: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: lib

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
MASK = "[MASK]"

In [4]:
names = [
    "Twitter/twhin-bert-base",
    "Geotrend/bert-base-pl-cased",
    "allegro/herbert-base-cased",
]

In [5]:
@dataclass
class Model:
    name: str
    model: AutoModelForMaskedLM
    tokenizer: AutoTokenizer


def predict(model: Model, text: str, n_predictions: int = 5):
    text = text.replace(MASK, model.tokenizer.mask_token)
    inputs = model.tokenizer(text, return_tensors="pt")

    mask_token_index = torch.where(
        inputs["input_ids"] == model.tokenizer.mask_token_id
    )[1][0].item()

    token_logits = model.model(**inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]

    return [
        text.replace(model.tokenizer.mask_token, model.tokenizer.decode([token]))
        for token in torch.topk(mask_token_logits, n_predictions)[-1]
    ]


In [6]:
models = [
    Model(
        name=name,
        model=AutoModelForMaskedLM.from_pretrained(name),
        tokenizer=AutoTokenizer.from_pretrained(name),
    ) for name in names
]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
sequences = [
        "Warszawa to największe [MASK].",  # Mianownik
        "Karol nie zbił [MASK].",  # Dopelnbiacz
        "Złodziej ukradł [MASK] torebkę.",  # Celownik
        "Od zimnego powietrza można nabawić się [MASK].",  # Biernik
        "Poluzowane śluby należy dokręcać [MASK].",  # Narzednik
        "W ostatnim czasie dużo mówi się o [MASK].",  # Miejscownik
        "Witaj drogi [MASK]!",  # Wolacz
    ]


In [8]:
def print_table(result):
    table = PrettyTable(result.keys())
    for i in range(len(result[list(result.keys())[0]])):
        row = [result[key][i] for key in result.keys()]
        table.add_row(row)
    print(table)
            

In [9]:
def predict_sequences(sequences):
    for sequence in sequences:
        result = {}
        for model in models:
            result[model.name] = predict(model, sequence)
        print(sequence)
        print_table(result)

## Przypadki

In [10]:
predict_sequences(sequences)

Warszawa to największe [MASK].
+---------------------------------+----------------------------------+----------------------------------+
|     Twitter/twhin-bert-base     |   Geotrend/bert-base-pl-cased    |    allegro/herbert-base-cased    |
+---------------------------------+----------------------------------+----------------------------------+
|  Warszawa to największe miasto. |  Warszawa to największe miasto.  |  Warszawa to największe miasto.  |
|  Warszawa to największe miasta. |  Warszawa to największe miasta.  | Warszawa to największe lotnisko. |
| Warszawa to największe miejsce. |   Warszawa to największe woj.    | Warszawa to największe centrum.  |
|   Warszawa to największe zło.   | Warszawa to największe Warszawa. |  Warszawa to największe miasta.  |
|  Warszawa to największe święto. |  Warszawa to największe miast.   | Warszawa to największe atrakcje. |
+---------------------------------+----------------------------------+----------------------------------+
Karol nie zbił 

In [11]:
sequences = [
    "Kocham mojego [MASK].",
    "Kocham moją [MASK].",
    "Kocham moje [MASK].",
]

In [12]:
predict_sequences(sequences)

Kocham mojego [MASK].
+-------------------------+-----------------------------+----------------------------+
| Twitter/twhin-bert-base | Geotrend/bert-base-pl-cased | allegro/herbert-base-cased |
+-------------------------+-----------------------------+----------------------------+
|   Kocham mojego kota.   |      Kocham mojego god.     |    Kocham mojego syna.     |
|    Kocham mojego psa.   |      Kocham mojego je.      |    Kocham mojego ojca.     |
|   Kocham mojego syna.   |      Kocham mojego dr.      |    Kocham mojego męża.     |
|  Kocham mojego miasta.  |      Kocham mojego itd.     |    Kocham mojego brata.    |
|   Kocham mojego konta.  |     Kocham mojego narod.    |    Kocham mojego synka.    |
+-------------------------+-----------------------------+----------------------------+
Kocham moją [MASK].
+-------------------------+-----------------------------+----------------------------+
| Twitter/twhin-bert-base | Geotrend/bert-base-pl-cased | allegro/herbert-base-cased |
+

In [13]:
sequences = [
    "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.",
    "Dzięki [MASK] wszystkie obiekty posiadające masę lub energię wzajemnie przyciągają się.",
    "Kierujący pojazdem przekroczył dozwoloną na polskich autostradach prędkość [MASK] kilometrów na godzinę.",
    ]

In [14]:
predict_sequences(sequences)

[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+----------------------------------------------------------------------------------------+
|                               Twitter/twhin-bert-base                                |                            Geotrend/bert-base-pl-cased                            |                               allegro/herbert-base-cased                               |
+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+----------------------------------------------------------------------------------------+
| Ukraina wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza. | Jego wrze w temperaturze 100 stopni, a za

**Which of the models produced the best results?**

W mojej ocenie z zadaniem predykcji najlpepiej poradził sobie model Allegro. 

**Was any of the models able to capture Polish grammar?**

W bardzo wielu przypadkach modele nie radziły sobie z tworzeniem poprawnych gramatycznie wyrazów, nawet w przypadkach tak prostych jak "Witaj drogi Łukasz!".

**Was any of the models able to capture long-distant relationships between the words?**

W zasadzie wszystkie modele poradziły sobie z dopasowaniem wyrazów o właściwym rodzaju. 

**Was any of the models able to capture world knowledge?**

None of evaluated models was able to capture real world knowledge. The Allegro model was indeed able to provide water as boiling in 100 degrees, althought presumably due to how common "water" and "boiling" are used together in Polish language. None of the models has finished the definition of gravitation, nor was any of them able to provide the maximum legal speed on polish highways. The output looks as if models have only tried to put a text that would fit visually to the rest of the sentence. 

**What are the most striking errors made by the models?**

Niektóre ze zdań otrzymanych z modeli nie mają logicznego sensu. Przykładowo model Twittera sformułował zdanie *Karol nie zbił wody* które znaczenia jeszcze nie udało mi się odkryć. 